In [8]:
from show_result import show_result
import os
for f in os.listdir("results/tests_1014")[::-1][:4]:
    print("\033[32m> {}\033[0m".format(f))
    show_result(f"results/tests_1014/{f}")
    print()

> exp_7_lstm_80_mask

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[     lstm] -	-	-	-	-	-	-	-	5.335	-	-	-

> exp_6_mpnn_lstm_80_mask

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[mpnn_lstm] -	-	-	-	-	-	-	-	5.314	-	-	-

> exp_5_lstm

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[     lstm] 5.333	2.318	-	-	5.777	2.031	-	-	5.335	2.084	-	-

> exp_4_missing_nodes_80_mask

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 5.539	-	-	-	-	-	-	-	6.161	-	-	-



In [2]:
raise NotImplementedError
import os

def delete_pth_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pth'):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# 使用示例
delete_pth_files('results/tests_old')


NotImplementedError: 